In [ ]:
from ib_async import *
import pandas as pd

util.startLoop()

ib = IB()
ib.connect('ib-gateway', 4004)

In [ ]:
# dont use https://www.asx.com.au/asx/research/ASXListedCompanies.csv as company can have secondary issues
# see https://www.asx.com.au/markets/market-resources/asx-codes-and-descriptors

# get all asx stock instruments
asx_stocks = []
try:
  asx_stocks = pd.read_csv("ibkr_asx_stocks.csv", comment='#')['localSymbol'].to_list()
except:
  import requests
  page = 1
  while page:
    post_data = {"pageNumber":page,"pageSize":"500","sortField":"symbol","sortDirection":"asc","productCountry":["AU"],"productSymbol":"","newProduct":"all","productType":["STK"],"domain":"hk"}
    response = requests.post("https://www.interactivebrokers.com/webrest/search/products-by-filters", json=post_data)
    response.raise_for_status()

    products = response.json()['products']
    asx_stocks.extend(products)
    page = page + 1 if len(products) else 0

  # these symbols fail to resolve
  excludes = [
    "1AGNA", "1STN", "3DANA", "AAUNB", "ACBN", "ACQN", "ADDNB", "ADSN", "AFWN", "AGH", "AGSN", "AJMN", "AKNN", "ALINA",
    "ALVNA", "ALYN", "AMSNA", "ANONC", "ANPNB", "ARCNB", "AS2N", "ASMN", "AUN", "AUNR", "AUQN", "AV1NA", "AVLDA", "AVRN",
    "AWVNA", "AYUHB", "BASNE", "BBCN", "BDG", "BENPD", "BEXN", "BITNE", "BLGDA", "BLGNA", "BM8.FAKE", "BMEN", "BNONA",
    "BNRNA", "BOAN", "BOD", "BOQPD", "BRUN", "BTRNB", "BUDNA", "BYH", "CAMPA", "CAPNB", "CDTNA", "CG1NA", "CHRCA", "CIONC",
    "CL8NE", "CNJND", "CSM", "CVSNA", "CVSNB", "CZNN", "DTRN", "DWG.OLD1", "DXNNB", "DY6", "DYLNC", "ECSNC", "EGNN", "EQN",
    "EQS", "EVEN", "EWCNA", "FBM", "FFXND", "FNXN", "FTZNA", "G11DA", "GBZ", "GLA", "GMDND", "GMEN", "GMNNA", "GMNNB", "GNMND",
    "GNMNF", "GSNNB", "GTGN", "HILNA", "HILNB", "HILR", "HNRNA", "HORNC", "ICGR", "ICNN", "IFM", "IMFHA", "INPN", "IPTNA",
    "KAUN", "KFENA", "KKONB", "KTGNB", "LGL", "LNU", "LOC", "LOMNE", "LPDNH", "LRSXE", "LSHN", "MATN", "MAYN", "MCTN", "MDR",
    "MEBN", "MEBNB", "MELDA", "MELNA", "MG1", "MGTNF", "MLMNC", "MOQN", "MOTNB", "MPXN", "MRCN", "MRCNA", "MXUPA", "NCZN",
    "NET", "NGYN", "NMRNB", "NORN", "NVUN", "NWENA", "NWFN", "NZS", "OLLNA", "OOKNA", "OPNN", "OVNNA", "OZZN", "PABN", "PAKNB",
    "PBHN", "PCIN", "PCKDB", "PDZN", "PEXND", "PGON", "PH2", "PHLN", "PLN", "PMC", "PRS", "PSLNB", "PXXNA", "RANN", "RBD",
    "RBRDA", "RD1NC", "RGLN", "RILN", "RWDN", "RYDNA", "SBKPA", "SBMN", "SENDA", "SLM", "SMRN", "SMXN", "SOVN", "STKN", "STKND",
    "SVWPA", "SW1", "SYAN", "THB", "TMG", "TMGNA", "TMHN", "TNRN", "TNRNA", "TONNF", "TPSN", "TRYNA", "TSODD", "TYMNA", "TZLNC",
    "UBIN", "UVA", "VFXDB", "VMSDC", "VORN", "VRXND", "VXRNF", "W2V", "WBE", "WBEN", "WBTN", "WBTNB", "WHANC", "WKTND", "WLD",
    "WLENA", "WMCNF", "WMXNE", "WRMNE", "WWGN", "WWIN", "XTEN", "YTMANZ", "YTMAZJ", "YTMF08", "YTMF10", "YTMF11", "YTMMGR",
    "YTMMQG", "YTMTLS", "YTMWB1"
  ]
  #todo should also append missing stocks such as
  asx_stocks_df = pd.DataFrame(asx_stocks)
  asx_stocks_df[~asx_stocks_df['localSymbol'].isin(excludes)].to_csv('ibkr_asx_stocks.csv', index=False)

In [ ]:
# missed/bad stock symbols list
missed_stocks = list(map(lambda x: {"localSymbol": x}, ["MEL"]))

for asx_stock in (asx_stocks + missed_stocks):
  contract = Stock(asx_stock["localSymbol"], "SMART", "AUD")

  dt = ""
  barsList = []
  while True:
    bars = ib.reqHistoricalData(
      contract,
      endDateTime=dt,
      durationStr="5 Y",
      barSizeSetting="1 day",
      whatToShow="TRADES",
      useRTH=True,
      formatDate=1)
    if not bars or bars[0].date == dt:
      break
    barsList.append(bars)
    dt = bars[0].date
    print(f"{contract.symbol} - {dt} ({len(bars)})")

  if not barsList:
    print("No data for " + contract.symbol)
  else:
    df = util.df([b for bars in reversed(barsList) for b in bars])
    df.to_csv(contract.symbol + '.csv', index=False)
